<a href="https://colab.research.google.com/github/agungtriu/Belajar-Pengembangan-Machine-Learning/blob/master/modul_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latihan Tokenization


In [64]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [65]:
tokenizer = Tokenizer(num_words=15, oov_token='-')

In [66]:
teks = ['Saya suka programming', 
        'Programming sangat menyenangkan!', 
        'Machine Learning berbeda dengan pemrograman konvensional']

In [67]:
tokenizer.fit_on_texts(teks)

In [68]:
sequences = tokenizer.texts_to_sequences(teks)

In [69]:
print(tokenizer.word_index)

{'-': 1, 'programming': 2, 'saya': 3, 'suka': 4, 'sangat': 5, 'menyenangkan': 6, 'machine': 7, 'learning': 8, 'berbeda': 9, 'dengan': 10, 'pemrograman': 11, 'konvensional': 12}


In [70]:
print(tokenizer.texts_to_sequences(['Saya suka programming']))
print(tokenizer.texts_to_sequences(['Saya suka belajar programming sejak SMP']))

[[3, 4, 2]]
[[3, 4, 1, 2, 1, 1]]


In [71]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences_samapanjang = pad_sequences(sequences)
print(sequences_samapanjang)

[[ 0  0  0  3  4  2]
 [ 0  0  0  2  5  6]
 [ 7  8  9 10 11 12]]


In [73]:
sequences_samapanjang = pad_sequences(sequences, 
                                      padding='post',
                                      maxlen=5)
print(sequences_samapanjang)

[[ 3  4  2  0  0]
 [ 2  5  6  0  0]
 [ 8  9 10 11 12]]


In [74]:
sequences_samapanjang = pad_sequences(sequences, 
                                      padding='post',
                                      maxlen=5,
                                      truncating='post')
print(sequences_samapanjang)

[[ 3  4  2  0  0]
 [ 2  5  6  0  0]
 [ 7  8  9 10 11]]


# Binary Text Classification

In [80]:
import pandas as pd
df = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df.tail()

,sentence,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


In [81]:
from sklearn.model_selection import train_test_split
kalimat = df['sentence'].values
y = df['label'].values
kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, test_size=0.2)

In [82]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=250, oov_token='x')
tokenizer.fit_on_texts(kalimat_latih) 
tokenizer.fit_on_texts(kalimat_test)
 
sekuens_latih = tokenizer.texts_to_sequences(kalimat_latih)
sekuens_test = tokenizer.texts_to_sequences(kalimat_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [83]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(250, 16, input_length=20),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [84]:
num_epochs = 30
history = model.fit(padded_latih, y_latih, epochs=num_epochs, 
                    validation_data=(padded_test, y_test), verbose=2)

Epoch 1/30
25/25 - 0s - loss: 0.6931 - accuracy: 0.5013 - val_loss: 0.6929 - val_accuracy: 0.4600
Epoch 2/30
25/25 - 0s - loss: 0.6914 - accuracy: 0.6075 - val_loss: 0.6907 - val_accuracy: 0.6450
Epoch 3/30
25/25 - 0s - loss: 0.6891 - accuracy: 0.6400 - val_loss: 0.6879 - val_accuracy: 0.6600
Epoch 4/30
25/25 - 0s - loss: 0.6858 - accuracy: 0.6150 - val_loss: 0.6833 - val_accuracy: 0.6450
Epoch 5/30
25/25 - 0s - loss: 0.6802 - accuracy: 0.6750 - val_loss: 0.6791 - val_accuracy: 0.6800
Epoch 6/30
25/25 - 0s - loss: 0.6723 - accuracy: 0.6800 - val_loss: 0.6686 - val_accuracy: 0.6850
Epoch 7/30
25/25 - 0s - loss: 0.6607 - accuracy: 0.6712 - val_loss: 0.6595 - val_accuracy: 0.7050
Epoch 8/30
25/25 - 0s - loss: 0.6444 - accuracy: 0.7075 - val_loss: 0.6425 - val_accuracy: 0.6850
Epoch 9/30
25/25 - 0s - loss: 0.6231 - accuracy: 0.7513 - val_loss: 0.6288 - val_accuracy: 0.7150
Epoch 10/30
25/25 - 0s - loss: 0.5976 - accuracy: 0.7812 - val_loss: 0.6067 - val_accuracy: 0.7200
Epoch 11/30
25/25 -

# Multiclass Text Classification

In [86]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [87]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [88]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [89]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [91]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [92]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [93]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 26s - loss: 1.6100 - accuracy: 0.2027 - val_loss: 1.6105 - val_accuracy: 0.1791
Epoch 2/30
26/26 - 30s - loss: 1.6031 - accuracy: 0.2164 - val_loss: 1.6048 - val_accuracy: 0.2438
Epoch 3/30
26/26 - 29s - loss: 1.5622 - accuracy: 0.3682 - val_loss: 1.5888 - val_accuracy: 0.2239
Epoch 4/30
26/26 - 32s - loss: 1.3508 - accuracy: 0.4465 - val_loss: 1.6073 - val_accuracy: 0.2786
Epoch 5/30
26/26 - 29s - loss: 1.1132 - accuracy: 0.5398 - val_loss: 1.7169 - val_accuracy: 0.2637
Epoch 6/30
26/26 - 28s - loss: 0.8527 - accuracy: 0.6903 - val_loss: 1.7157 - val_accuracy: 0.3582
Epoch 7/30
26/26 - 28s - loss: 0.5848 - accuracy: 0.8445 - val_loss: 1.9745 - val_accuracy: 0.3234
Epoch 8/30
26/26 - 28s - loss: 0.2605 - accuracy: 0.9490 - val_loss: 2.9079 - val_accuracy: 0.2836
Epoch 9/30
26/26 - 28s - loss: 0.1450 - accuracy: 0.9614 - val_loss: 2.7399 - val_accuracy: 0.3134
Epoch 10/30
26/26 - 29s - loss: 0.0541 - accuracy: 0.9925 - val_loss: 3.2457 - val_accuracy: 0.3035
Epoch 11/